In [4]:
# exposures = np.arange(432, 440)

# GP2D.batch(exposures, sample, nSigma, test_frac, datafile, utheta0, vtheta0, bounds)

In [1]:
import gpr

import numpy as np

In [2]:
utheta0 = np.array([50, .1, .1, 0, 1.4])
vtheta0 = np.array([50, .1, .1, 0, 1.4])
bounds = np.array([(1e-4, 1e5), (1e-4, 100), (1e-4, 100), (-4*np.pi, 4*np.pi), (1e-5, 100)])

exposures = np.arange(400, 500)
sample = np.array([-0.3, 0.3, -0.3, 0.3])
# sample = np.array([-0.3, 0, 0, 0.3])
# sample = None
nSigma = 4
test_frac = 0.50
datafile = 'hoid'
nExposure = 500

In [3]:
GP2D = gpr.calcGPR(nLML_print=False)
GP2D.genGPR()
GP2D.load(sample, nSigma, test_frac, datafile=datafile, nExposure=nExposure)
GP2D.solve2d(utheta0, vtheta0, bounds, verbose=False)
GP = GP2D.fit2d(GP2D.u_result.x, GP2D.v_result.x)

In [21]:
import astropy.io.fits as fits
GP.__dict__.keys()

dict_keys(['coord', 'verbose', 'nLML_factor', 'nLML_print', 'random_state', 'rng', 'eps', 'synth', 'datafile', 'nExposure', 'fits', 'exposure', 'sample', 'X', 'Y', 'E', 'Xtrain', 'Xtest', 'Ytrain', 'Ytest', 'Etrain', 'Etest', 'nTrain', 'nTest', 'nParams', 'u_theta', 'u_K', 'u_Kss', 'u_Ks', 'u_W', 'u_L', 'u_alpha', 'u_fbar_s', 'u_v', 'u_V_s', 'u_sigma', 'u_nLML', 'fbar_s', 'sigma', 'nLML', 'v_theta', 'RSS', 'chisq', 'red_chisq'])

In [25]:
GP.random_state
GP.eps
Gp.datafile
GP.nExposure
GP.sample
GP.X, GP.Xtrain, GP.Xtest
GP.Y, GP.Ytrain, GP.Ytest
GP.E, GP.Etrain, GP.Etest
GP.nTrain
GP.nTest

GP.fbar_s
GP.sigma

GP.u_theta



GP.u_nLML

GP.v_theta
GP.v_fbar_s
GP.v_V_s
GP.v_sigma
GP.v_nLML

GP.RSS
GP.chisq
GP.red_chisq

array(<mtrand.RandomState object at 0x7f59b3480c18>, dtype=object)

In [28]:
np.array(GP.rng)

array(<mtrand.RandomState object at 0x7f59b3480c18>, dtype=object)